<a href="https://colab.research.google.com/github/derek-shing/Kafka_Python/blob/master/Kafka_Python_for_recipe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The first program we are going to write is the producer. It will access Allrecpies.com and fetch the raw HTML and store in raw_recipes topic.

https://towardsdatascience.com/getting-started-with-apache-kafka-in-python-604b3250aa05

In [0]:
import requests
from bs4 import BeautifulSoup
from time import sleep
from kafka import KafkaConsumer, KafkaProducer


In [3]:
pip install kafka-python

     |████████████████████████████████| 266kB 2.8MB/s 


In [0]:
def fetch_raw(recipe_url):
    html = None
    print('Processing..{}'.format(recipe_url))
    try:
        r = requests.get(recipe_url)
        if r.status_code == 200:
            html = r.text
    except Exception as ex:
        print('Exception while accessing raw html')
        print(str(ex))
    finally:
        return html.strip()

In [0]:
def get_recipes():
    recipies = []
    salad_url = 'https://www.allrecipes.com/recipes/96/salad/'
    url = 'https://www.allrecipes.com/recipes/96/salad/'
    print('Accessing list')

    try:
        r = requests.get(url)
        if r.status_code == 200:
            html = r.text
            soup = BeautifulSoup(html, 'lxml')
            links = soup.select('.fixed-recipe-card__h3 a')
            idx = 0
            for link in links:

                sleep(2)
                recipe = fetch_raw(link['href'])
                recipies.append(recipe)
                idx += 1
                if idx > 2:
                    break
    except Exception as ex:
        print('Exception in get_recipes')
        print(str(ex))
    finally:
        return recipies

In [12]:
recipes = (get_recipes())

Accessing list
Processing..https://www.allrecipes.com/recipe/52234/alligator-pears-and-apples/
Processing..https://www.allrecipes.com/recipe/270850/roasted-beet-salad/
Processing..https://www.allrecipes.com/recipe/215030/taco-slaw/


In [13]:
len(recipes)

3

In [14]:
recipes[0]

'<!DOCTYPE html>\r\n<html lang="en-us">\r\n<head>\r\n    <title>Alligator Pears and Apples Recipe - Allrecipes.com</title>\r\n\r\n<script src=\'https://secureimages.allrecipes.com/assets/deployables/v-1.179.0.5174/karma.bundled.js\' async=true></script>\r\n\r\n\r\n    <!--Make our website baseUrl available to the client-side code-->\r\n    <script type="text/javascript">\r\n        var AR = AR || {};\r\n\r\n        AR.segmentWriteKey = "RnmsxUrjIjM7W62olfjKgJrcsVlxe68V";\r\n        AR.baseWebsiteUrl = \'https://www.allrecipes.com\';\r\nwindow.dataLayer={"version":"1.0","pageInstanceId":"www.allrecipes.com/recipe/52234/alligator-pears-and-apples/","externalLinkId":"","page":{"pageInfo":{"pageId":"52234","pageName":"/recipe/52234/alligator-pears-and-apples/","destinationUrl":"https://www.allrecipes.com/recipe/52234/alligator-pears-and-apples/","sysEnv":"RD0003FFB37290","variant":"Control","abTestName":"RDP_Video_Step","version":"20180321","issueDate":"11/07/2019 07:31:11","effectiveDate"

In [0]:
soup = BeautifulSoup(recipes[0])

In [0]:
soup

In [0]:
def publish_message(producer_instance, topic_name, key, value):
    try:
        key_bytes = bytes(key, encoding='utf-8')
        value_bytes = bytes(value, encoding='utf-8')
        producer_instance.send(topic_name, key=key_bytes, value=value_bytes)
        producer_instance.flush()
        print('Message published successfully.')
    except Exception as ex:
        print('Exception in publishing message')
        print(str(ex))


def connect_kafka_producer():
    _producer = None
    try:
        _producer = KafkaProducer(bootstrap_servers=['54.153.60.227:9092'], api_version=(0, 10))
    except Exception as ex:
        print('Exception while connecting Kafka')
        print(str(ex))
    finally:
        return _producer

In [9]:
all_recipes = get_recipes()
if len(all_recipes) > 0:
        kafka_producer = connect_kafka_producer()
        for recipe in all_recipes:
            publish_message(kafka_producer, 'raw_recipes', 'raw', recipe.strip())
        if kafka_producer is not None:
            kafka_producer.close()

Accessing list
Processing..https://www.allrecipes.com/recipe/228340/kale-and-feta-salad/
Processing..https://www.allrecipes.com/recipe/270850/roasted-beet-salad/
Processing..https://www.allrecipes.com/recipe/215030/taco-slaw/
Message published successfully.
Message published successfully.
Message published successfully.
